In [1]:
#!/usr/bin/env python3

import numpy as np
import pandas as pd
import xarray as xr
import zipfile
import glob
import netCDF4 as nc
import xesmf as xe
from scipy import stats

def trend_finder(timeseries_maps):
    """
    Find decadal trends at each lat and lon point. 
    """
    shape_of_maps = np.shape(timeseries_maps)
    # must be adjusted based on len of time used for trends
    years = np.linspace(0,3.5,36)
    trend_map = []
    for lat in range(0, shape_of_maps[1]):
        lat_line = []
        for lon in range(0, shape_of_maps[2]):
            timeseries = timeseries_maps[:,lat,lon]
            trend = stats.linregress(x=years, y=timeseries)[0] # in K/dec
            lat_line.append(trend)
        trend_map.append(lat_line)
    return(np.array(trend_map))
            
# load path to CMIP 6 data
path_to_CMIP6_data = '/home/disk/pna2/aodhan/CMIP6/historical_monthly_psl_google/*'
model_paths = glob.glob(path_to_CMIP6_data)

# here we select the start times of trends, this can be changed
start_times = np.arange(1854,1980,5)

# for each model...
for model in model_paths[-2:-1]: # the -1 is because the last "model" is the directory /trendmap where data is stored
    realizations = glob.glob(model + '/*')

    # create data dictionary
    model_trends = []
    
    # for each realization...
    simulation_index = []
    simulation_count = 1
    for ensemble_member in realizations:

        # if we run into problems with given simulation, skip it
        simulation = xr.open_dataset(ensemble_member)
        model_name = simulation.source_id
        simulation_name = simulation.variant_label
        simulation_number = int(simulation_name.split('i')[0][1:])

        # CMIP6 models must be regridded, below we define input and output grids
        latitudes = simulation.lat.values
        longitudes = simulation.lon.values
        InputGrid = {"lon": longitudes, "lat": latitudes}
        OutputGrid = {"lon": np.arange(1.25, 358.751, 2.5), "lat": np.arange(-88.75, 88.751, 2.5)}
        regridder = xe.Regridder(InputGrid, OutputGrid, "bilinear")
        
        # create lists for where to append data
        all_timeperiod_trends = []
        time_keys = []
        
        try:
            # find time slices for just winter
            for i in start_times:
                start_date = str(i) + '-01'
                end_date = str(i+35) + '-12'

                # find correct time period of data
                time_slice_data = simulation.sel(time=slice(start_date, end_date))
                data_array = time_slice_data.psl.values

                # find trends for each of the time periods
                shape_of_data_array = np.shape(data_array)
                data_calendar = np.reshape(data_array, (36, 12, shape_of_data_array[1], shape_of_data_array[2]))
                season_calendar = [data_calendar[:,10],data_calendar[:,11], data_calendar[:,0], 
                                data_calendar[:,1], data_calendar[:,2]]
                timeseries_map = np.nanmean(season_calendar, axis=0)
                trend_map = trend_finder(timeseries_map)
                trend_map_2p5x2p5 = regridder(trend_map)
                all_timeperiod_trends.append(trend_map_2p5x2p5)
                time_keys.append(i)

            # if you can successfully create trend maps, then add this index as a simulation used
            simulation_index.append(simulation_number)
            print(model_name, simulation_name, simulation_number)
        except:
            print('skipping: ', simulation_name)
            continue
    
        # append all timeperiod trends 
        model_trends.append(all_timeperiod_trends)

    # Timeperiod data will be dumped into NetCDF files
    fileName = path_to_CMIP6_data[:-2] + '/trendmap/' + model_name.replace("-", "_") + '_PSL_NDJFM_TrendMaps.nc'

    # Create netcdf file with dimensions
    ds = nc.Dataset(fileName, 'w', format='NETCDF4')
    ensemble_member = ds.createDimension('ensemble_member', len(simulation_index))
    TrendTimePeriod = ds.createDimension('TrendTimePeriod', len(time_keys)) # 26 timeperiods
    Lat = ds.createDimension('Lat', 72)
    Lon = ds.createDimension('Lon', 144)

    # Add variables to dimensions
    ensemble_member = ds.createVariable('ensemble_member', int, ('ensemble_member',))
    TrendTimePeriod = ds.createVariable('TrendTimePeriod', int, ('TrendTimePeriod',))
    Lat = ds.createVariable('Lat', 'f4', ('Lat',))
    Lon = ds.createVariable('Lon', 'f4', ('Lon',))
    Ts_trends = ds.createVariable('ts_trend', 'f4', ('ensemble_member', 'TrendTimePeriod', 'Lat', 'Lon'))

    # Assing values to variables
    ensemble_member[:] = simulation_index
    TrendTimePeriod[:] = time_keys
    Lat[:] = np.arange(-88.75, 88.751, 2.5)
    Lon[:] = np.arange(1.25, 358.751, 2.5)
    Ts_trends[:] = model_trends

    ds.close()


EC-Earth3 r9i1p1f1 9
EC-Earth3 r11i1p1f1 11
EC-Earth3 r13i1p1f1 13
EC-Earth3 r15i1p1f1 15
EC-Earth3 r6i1p1f1 6
EC-Earth3 r1i1p1f1 1
EC-Earth3 r20i1p1f1 20
skipping:  r118i1p1f1
skipping:  r127i1p1f1
skipping:  r126i1p1f1
skipping:  r119i1p1f1
skipping:  r122i1p1f1
skipping:  r120i1p1f1
skipping:  r121i1p1f1
skipping:  r106i1p1f1
skipping:  r149i1p1f1
skipping:  r105i1p1f1
skipping:  r103i1p1f1
skipping:  r102i1p1f1
skipping:  r150i1p1f1
skipping:  r104i1p1f1
skipping:  r101i1p1f1
skipping:  r128i1p1f1
skipping:  r129i1p1f1
skipping:  r130i1p1f1
skipping:  r133i1p1f1
skipping:  r134i1p1f1
skipping:  r136i1p1f1
skipping:  r137i1p1f1
skipping:  r145i1p1f1
skipping:  r138i1p1f1
skipping:  r131i1p1f1
skipping:  r132i1p1f1
skipping:  r135i1p1f1
skipping:  r109i1p1f1
skipping:  r108i1p1f1
skipping:  r107i1p1f1
skipping:  r117i1p1f1
skipping:  r116i1p1f1
skipping:  r115i1p1f1
skipping:  r124i1p1f1
skipping:  r123i1p1f1
skipping:  r125i1p1f1
skipping:  r111i1p1f1
skipping:  r110i1p1f1
skipping:

In [3]:
#!/usr/bin/env python3

import numpy as np
import pandas as pd
import xarray as xr
import zipfile
import glob
import netCDF4 as nc
import xesmf as xe
from scipy import stats

def trend_finder(timeseries_maps):
    """
    Find decadal trends at each lat and lon point. 
    """
    shape_of_maps = np.shape(timeseries_maps)
    # must be adjusted based on len of time used for trends
    years = np.linspace(0,3.5,36)
    trend_map = []
    for lat in range(0, shape_of_maps[1]):
        lat_line = []
        for lon in range(0, shape_of_maps[2]):
            timeseries = timeseries_maps[:,lat,lon]
            trend = stats.linregress(x=years, y=timeseries)[0] # in K/dec
            lat_line.append(trend)
        trend_map.append(lat_line)
    return(np.array(trend_map))
            
# load path to CMIP 6 data
path_to_CMIP6_data = '/home/disk/pna2/aodhan/CMIP6/historical_monthly_tas_google/*'
model_paths = glob.glob(path_to_CMIP6_data)

# here we select the start times of trends, this can be changed
start_times = np.arange(1854,1980,5)

# for each model...
for model in model_paths[-2:-1]:
    
    realizations = glob.glob(model + '/*')

    # create data dictionary
    model_trends = []
    
    # for each realization...
    simulation_index = []
    for ensemble_member in realizations:
        simulation = xr.open_dataset(ensemble_member)
        model_name = simulation.source_id
        simulation_name = simulation.variant_label
        simulation_number = int(simulation_name.split('i')[0][1:])
        print(model_name, simulation_name, simulation_number)

         # CMIP6 models must be regridded, below we define input and output grids
        latitudes = simulation.lat.values
        longitudes = simulation.lon.values
        InputGrid = {"lon": longitudes, "lat": latitudes}
        OutputGrid = {"lon": np.arange(1.25, 358.751, 2.5), "lat": np.arange(-88.75, 88.751, 2.5)}
        regridder = xe.Regridder(InputGrid, OutputGrid, "bilinear", periodic=True)
        
        # create lists for where to append data
        all_timeperiod_trends = []
        time_keys = []

        try:
            # find time slices for just winter
            for i in start_times:
                start_date = str(i) + '-01'
                end_date = str(i+35) + '-12'

                # find correct time period of data
                time_slice_data = simulation.sel(time=slice(start_date, end_date))
                data_array = time_slice_data.tas.values

                # find trends for each of the time periods
                shape_of_data_array = np.shape(data_array)
                data_calendar = np.reshape(data_array, (36, 12, shape_of_data_array[1], shape_of_data_array[2]))
                season_calendar = [data_calendar[:,10],data_calendar[:,11], data_calendar[:,0], 
                                data_calendar[:,1], data_calendar[:,2]]
                timeseries_map = np.nanmean(season_calendar, axis=0)
                trend_map = trend_finder(timeseries_map)
                trend_map_2p5x2p5 = regridder(trend_map)
                all_timeperiod_trends.append(trend_map_2p5x2p5)
                time_keys.append(i)

            # if you can successfully create trend maps, then add this index as a simulation used
            simulation_index.append(simulation_number)
        except:
            print('skipping: ', simulation_name)
            continue
        
        # append all timeperiod trends 
        model_trends.append(all_timeperiod_trends)

    # Timeperiod data will be dumped into NetCDF files
    fileName = path_to_CMIP6_data[:-2] + '/trendmap/' + model_name.replace("-", "_") + '_TAS_NDJFM_TrendMaps.nc'

    # Create netcdf file with dimensions
    ds = nc.Dataset(fileName, 'w', format='NETCDF4')
    ensemble_member = ds.createDimension('ensemble_member', len(simulation_index))
    TrendTimePeriod = ds.createDimension('TrendTimePeriod', len(time_keys)) # 26 timeperiods
    Lat = ds.createDimension('Lat', 72)
    Lon = ds.createDimension('Lon', 144)

    # Add variables to dimensions
    ensemble_member = ds.createVariable('ensemble_member', int, ('ensemble_member',))
    TrendTimePeriod = ds.createVariable('TrendTimePeriod', int, ('TrendTimePeriod',))
    Lat = ds.createVariable('Lat', 'f4', ('Lat',))
    Lon = ds.createVariable('Lon', 'f4', ('Lon',))
    Ts_trends = ds.createVariable('ts_trend', 'f4', ('ensemble_member', 'TrendTimePeriod', 'Lat', 'Lon'))

    # Assing values to variables
    ensemble_member[:] = simulation_index
    TrendTimePeriod[:] = time_keys
    Lat[:] = np.arange(-88.75, 88.751, 2.5)
    Lon[:] = np.arange(1.25, 358.751, 2.5)
    Ts_trends[:] = model_trends

    ds.close()


EC-Earth3 r9i1p1f1 9
EC-Earth3 r11i1p1f1 11
EC-Earth3 r13i1p1f1 13
EC-Earth3 r15i1p1f1 15
EC-Earth3 r6i1p1f1 6
EC-Earth3 r1i1p1f1 1
EC-Earth3 r119i1p1f1 119
skipping:  r119i1p1f1
EC-Earth3 r118i1p1f1 118
skipping:  r118i1p1f1
EC-Earth3 r117i1p1f1 117
skipping:  r117i1p1f1
EC-Earth3 r127i1p1f1 127
skipping:  r127i1p1f1
EC-Earth3 r125i1p1f1 125
skipping:  r125i1p1f1
EC-Earth3 r126i1p1f1 126
skipping:  r126i1p1f1
EC-Earth3 r120i1p1f1 120
skipping:  r120i1p1f1
EC-Earth3 r121i1p1f1 121
skipping:  r121i1p1f1
EC-Earth3 r122i1p1f1 122
skipping:  r122i1p1f1
EC-Earth3 r148i1p1f1 148
skipping:  r148i1p1f1
EC-Earth3 r106i1p1f1 106
skipping:  r106i1p1f1
EC-Earth3 r149i1p1f1 149
skipping:  r149i1p1f1
EC-Earth3 r105i1p1f1 105
skipping:  r105i1p1f1
EC-Earth3 r103i1p1f1 103
skipping:  r103i1p1f1
EC-Earth3 r101i1p1f1 101
skipping:  r101i1p1f1
EC-Earth3 r150i1p1f1 150
skipping:  r150i1p1f1
EC-Earth3 r104i1p1f1 104
skipping:  r104i1p1f1
EC-Earth3 r102i1p1f1 102
skipping:  r102i1p1f1
EC-Earth3 r136i1p1f1 1

PermissionError: [Errno 13] Permission denied: b'/home/disk/pna2/aodhan/CMIP6/historical_monthly_tas_google/trendmap/EC_Earth3_TAS_NDJFM_TrendMaps.nc'

In [11]:
# Timeperiod data will be dumped into NetCDF files
fileName = path_to_CMIP6_data[:-2] + '/trendmap/' + model_name.replace("-", "_") + '_TAS_NDJFM_TrendMaps.nc'

# Create netcdf file with dimensions
ds = nc.Dataset(fileName, 'w', format='NETCDF4')
ensemble_member = ds.createDimension('ensemble_member', len(simulation_index))
TrendTimePeriod = ds.createDimension('TrendTimePeriod', len(time_keys)) # 26 timeperiods
Lat = ds.createDimension('Lat', 72)
Lon = ds.createDimension('Lon', 144)

# Add variables to dimensions
ensemble_member = ds.createVariable('ensemble_member', int, ('ensemble_member',))
TrendTimePeriod = ds.createVariable('TrendTimePeriod', int, ('TrendTimePeriod',))
Lat = ds.createVariable('Lat', 'f4', ('Lat',))
Lon = ds.createVariable('Lon', 'f4', ('Lon',))
Ts_trends = ds.createVariable('ts_trend', 'f4', ('ensemble_member', 'TrendTimePeriod', 'Lat', 'Lon'))

# Assing values to variables
ensemble_member[:] = simulation_index
TrendTimePeriod[:] = time_keys
Lat[:] = np.arange(-88.75, 88.751, 2.5)
Lon[:] = np.arange(1.25, 358.751, 2.5)
Ts_trends[:] = model_trends

ds.close()

In [ ]:
print('hi')